<a href="https://colab.research.google.com/github/SreeBhagya-S/Conference_22/blob/main/81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.8 MB/s 
     |████████████████████████████████| 596 kB 67.5 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install pytorch-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 8.0 MB/s 
     |████████████████████████████████| 880 kB 67.1 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 132 kB 67.1 MB/s 
     |████████████████████████████████| 8.9 MB 55.0 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 138 kB 5.5 MB/s 
     |████████████████████████████████| 127 kB 55.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=26fdb132a548c3e510238c9871db72cb80f1443f63f2ab97bad6fb99d209bee6
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras import utils as np_utils
from keras.utils.np_utils import to_categorical

import transformers
from transformers import AutoTokenizer,DistilBertModel, DistilBertConfig
from transformers import TFAutoModel

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Downloading datasets
!rm -rf t*
!wget https://raw.githubusercontent.com/SreeBhagya-S/Conference_22/main/manglish_test.tsv;
!wget https://raw.githubusercontent.com/SreeBhagya-S/Conference_22/main/manglish_train.csv;

In [ ]:
df = pd.read_csv('/content/manglish_train.csv',  sep=',', header=0)
df_test = pd.read_csv('/content/manglish_test.tsv',  sep='\t', header=0)

In [ ]:
df = df.dropna()

df_test = df_test.dropna()


In [ ]:
df

In [ ]:
from bs4 import BeautifulSoup

import nltk, string, re, spacy,unicodedata, random
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = text.strip()
    text = re.sub(r"http\S+", "", text, flags=re.MULTILINE)  # removing urls
    text = re.sub(r"www\S+", "", text, flags=re.MULTILINE)  # removing urls
    text = re.sub(r"&\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"\.{2,}", ".", text, flags=re.MULTILINE)  # continous dots
    text = re.sub(r"&", " and ", text)

    for i in [8221, 35, 36, 94, 8217]:
        text = text.replace(chr(i), "")
    text = text.replace("\n", ".")
    text = text.replace("&amp", " ")
    text = text.replace("-", "")
    text = text.replace(";", "")
    text = text.replace("@", "")
    text = text.replace('[{}]'.format(string.punctuation), '') #removing punctuations
    punctuation = '!!"$%&()*+-/:;<=>?[\\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = BeautifulSoup(text, 'html.parser').get_text()
      # Stopword Removing using nltk
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
   # tokens = [token.strip() for token in tokens if token not in STOP_WORDS]
    text = ' '.join(ch for ch in tokens)
    return text

In [ ]:
def process_data(df):
    df["text"] = df["text"].map(lambda x: preprocess_text(x))
    return df

In [ ]:
df = process_data(df)
df_test = process_data(df_test)

In [ ]:
df

In [ ]:
labels = ['Mixed_feelings','Negative','Positive','not-malayalam ','unknown_state']


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
senti = pd.value_counts(df['category'], sort=True)
senti.plot(kind="pie", labels=["Positive","unknown_state","not-malayalam","Negative","Mixed_feelings"], autopct="%.f%%")

plt.title("Review Sentiment Distribution")
plt.ylabel("")
plt.show()

In [ ]:
X_train = df.text
X_test = df_test.text

y_train = df.category
y_test = df_test.category

In [ ]:
train_lens = [len(i.split()) for i in X_train]
plt.hist(train_lens)

In [ ]:
y_train.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_test.shape

In [ ]:
y_test.value_counts()

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)

y_train = encoder.transform(y_train)
encoder.fit(y_test)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Tokenization of the data
def text_encode(text, tokenizer, max_len=100):
    tokens = text.apply(lambda x: tokenizer(x,return_tensors='tf', 
                                            truncation=True,
                                            padding='max_length',
                                            max_length=max_len, 
                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)

    return [input_ids,attention_mask]

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
   '''
   input_word_ids = tf.keras.Input(shape=(64,),dtype='int32')
   attention_masks = tf.keras.Input(shape=(64,),dtype='int32')

   sequence_output = bert_model([input_word_ids,attention_masks])
   output = sequence_output[1]
   output = tf.keras.layers.Dense(32,activation='relu')(output)
   output = tf.keras.layers.Dropout(0.2)(output)
   output = tf.keras.layers.Dense(1,activation='sigmoid')(output)

   model = tf.keras.models.Model(inputs = [input_word_ids,attention_masks], outputs = output)
   model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
   '''

In [ ]:
# model creation
def build_model(b_model, maxlen):
   #input_ids = tf.keras.Input(shape=(maxlen,),dtype=tf.int32, name='input_ids')
   #attention_mask = tf.keras.Input(shape=(maxlen,),dtype=tf.int32, name='attention_mask')
   input_ids = tf.keras.Input(shape=(maxlen,),dtype=tf.int32, name='input_ids')
   attention_mask = tf.keras.Input(shape=(maxlen,),dtype=tf.int32, name='attention_mask')
   sequence_output = b_model(input_ids,attention_mask=attention_mask)
   output = sequence_output[0][:,0,:]
   output = tf.keras.layers.Dense(32,activation='relu')(output)

   output = tf.keras.layers.Dropout(0.2)(output)
   output = tf.keras.layers.Dense(5,activation='softmax')(output)

   model = tf.keras.models.Model(inputs = [input_ids,attention_mask], outputs = [output])
   early_stopping = EarlyStopping(monitor='val_loss',patience=5,verbose=1)
   model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
   
   return model

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

**XLM**

In [ ]:
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig

In [ ]:
# Creating tokenizer
tokenizer_xlm = AutoTokenizer.from_pretrained('xlm-clm-enfr-1024')
pt_xlm = TFAutoModel.from_pretrained('xlm-clm-enfr-1024')

In [ ]:
X_train_input_ids, X_train_attention_mask = text_encode(X_train, tokenizer_xlm, max_len=100)
X_test_input_ids, X_test_attention_mask = text_encode(X_test, tokenizer_xlm, max_len=100)

In [ ]:
x_model = build_model(pt_xlm, maxlen=100)

In [ ]:
history = x_model.fit(
    [X_train_input_ids, X_train_attention_mask],
    y_train,
    batch_size=32,
    validation_data=([X_test_input_ids, X_test_attention_mask], y_test),
    epochs=10, callbacks=[EarlyStopping(monitor='val_loss',patience=5,verbose=1)]
)

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
y_predicted_xlm = x_model.predict([X_test_input_ids, X_test_attention_mask])

In [ ]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_predicted_xlm,axis=1), target_names=labels))

**XLNET**

In [ ]:
from transformers import XLNetTokenizer, XLNetModel

In [ ]:
# Creating tokenizer
tokenizer_xlnet = AutoTokenizer.from_pretrained('xlnet-base-cased')
pt_xlnet = TFAutoModel.from_pretrained('xlnet-base-cased')

In [ ]:
X_train_input_ids, X_train_attention_mask = text_encode(X_train, tokenizer_xlnet, max_len=100)
X_test_input_ids, X_test_attention_mask = text_encode(X_test, tokenizer_xlnet, max_len=100)

In [ ]:
xt_model = build_model(pt_xlnet, maxlen=100)

In [ ]:
history = xt_model.fit(
    [X_train_input_ids, X_train_attention_mask],
    y_train,
    batch_size=32,
    validation_data=([X_test_input_ids, X_test_attention_mask], y_test),
    epochs=10,callbacks=[EarlyStopping(monitor='val_loss',patience=5,verbose=1)]
)

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
y_predicted_xlnet = xt_model.predict([X_test_input_ids, X_test_attention_mask])

In [ ]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_predicted_xlnet,axis=1), target_names=labels))

**mBert**

In [ ]:
from transformers.models import bert
from transformers import BertTokenizer

In [ ]:
# Creating tokenizer
#tokenizer_bert = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

pt_bert = TFAutoModel.from_pretrained('bert-base-multilingual-cased')

In [ ]:
for layer in pt_bert.layers:
      layer.trainable = True

In [ ]:
X_train_input_ids, X_train_attention_mask = text_encode(X_train, tokenizer_bert, max_len=100)
X_test_input_ids, X_test_attention_mask = text_encode(X_test, tokenizer_bert, max_len=100)

In [ ]:
b_model = build_model(pt_bert, maxlen=100)

In [ ]:
history = b_model.fit(
    [X_train_input_ids, X_train_attention_mask],
    y_train,
    batch_size=32,
    validation_data=([X_test_input_ids, X_test_attention_mask], y_test),
    epochs=10,callbacks=[EarlyStopping(monitor='val_loss',patience=5,verbose=1)]
)

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
y_predicted_bert = b_model.predict([X_test_input_ids, X_test_attention_mask])

In [ ]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_predicted_bert,axis=1), target_names=labels))

**Distil Bert**

In [ ]:
from transformers.models import distilbert
from transformers import DistilBertTokenizer

In [ ]:
tokenizer_dbert  = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [ ]:
pt_dbert = TFAutoModel.from_pretrained('distilbert-base-multilingual-cased')

In [ ]:
for layer in pt_dbert.layers:
      layer.trainable = True

In [ ]:
X_train_input_ids, X_train_attention_mask = text_encode(X_train, tokenizer_dbert, max_len=100)
X_test_input_ids, X_test_attention_mask = text_encode(X_test, tokenizer_dbert, max_len=100)

In [ ]:
d_model = build_model(pt_dbert, maxlen=100)

In [ ]:
history = d_model.fit(
    [X_train_input_ids, X_train_attention_mask],
    y_train,
    batch_size=32,
    validation_data=([X_test_input_ids, X_test_attention_mask], y_test),
    epochs=10,callbacks=[EarlyStopping(monitor='val_loss',patience=5,verbose=1)]
)

In [ ]:
loss, accuracy = d_model.evaluate([X_test_input_ids, X_test_attention_mask], y_test)
print('Test accuracy :', accuracy*100,"%")

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
y_predicted_dbert = d_model.predict([X_test_input_ids, X_test_attention_mask])
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_predicted_dbert,axis=1), target_names=labels))

**RoBERTa**

In [ ]:
from transformers.models import roberta

In [ ]:
# Creating tokenizer
tokenizer_rbert = AutoTokenizer.from_pretrained('roberta-base')
pt_rbert = TFAutoModel.from_pretrained('roberta-base')

In [ ]:
for layer in pt_rbert.layers:
      layer.trainable = True

In [ ]:
X_train_input_ids, X_train_attention_mask = text_encode(X_train, tokenizer_rbert, max_len=100)
X_test_input_ids, X_test_attention_mask = text_encode(X_test, tokenizer_rbert, max_len=100)

In [ ]:
r_model = build_model(pt_rbert, maxlen=100)

In [ ]:
history = r_model.fit(
    [X_train_input_ids, X_train_attention_mask],
    y_train,
    batch_size=32,
    validation_data=([X_test_input_ids, X_test_attention_mask], y_test),
    epochs=20,callbacks=[EarlyStopping(monitor='val_loss',patience=5,verbose=1)]
)

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
y_predicted_rbert = r_model.predict([X_test_input_ids, X_test_attention_mask])

In [ ]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_predicted_rbert,axis=1), target_names=labels))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

def plot_AUC_ROC(y_true, y_pred):
    n_classes = 5

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
############################################################################################
    lw = 2
    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    
    plt.plot(fpr["micro"], tpr["micro"],
            label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
            color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
            label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
            color='navy', linestyle=':', linewidth=4)
    
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    classes_list1 = labels
    for i, color,c in zip(range(n_classes), colors,classes_list1):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                label='{0} (AUC = {1:0.2f})'
                ''.format(c, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_AUC_ROC(labels,y_pred__rbert)
plt.savefig("/content/sample_data/ROC_CODE_MIXED.pdf", format = 'pdf', dpi =1000, bbox_inches = 'tight')